# Transform the CSV

In [39]:
import pandas as pd
import json

# Define the file paths
csv_file = 'Niederscherli_raw.csv'
json_file = 'transform.json'
output_file = 'Niederscherli_transformed.csv'

# Read in the CSV file and the JSON file
df = pd.read_csv(csv_file)
with open(json_file, 'r') as f:
    config = json.load(f)

# Total row count
total_row_count = len(df)

match_count = 0

# Apply each condition/action pair from the JSON file
for index, row in df.iterrows():
    for condition_group in config['condition_groups']:
        matched_all_conditions = True
        for condition in condition_group['conditions']:
            column = condition['column']
            values = condition['value'] if isinstance(condition['value'], list) else [condition['value']]
            operator = condition.get('operator', 'equals')  # Defaults to 'equals' if no operator provided

            if operator == 'equals':
                if row[column] not in values:
                    matched_all_conditions = False
                    break
            elif operator == 'contains':
                if not any(value in str(row[column]) for value in values):
                    matched_all_conditions = False
                    break

        if matched_all_conditions:
            match_count += 1
            if 'actions' in condition_group:  # Check if 'actions' key exists
                for action in condition_group['actions']:
                    if action['type'] == 'set':
                        df.at[index, action['target_column']] = action['value']
                    elif action['type'] == 'copy':
                        df.at[index, action['target_column']] = row[action['source_column']]

# Save the updated DataFrame to a new CSV file
df.to_csv(output_file, index=False)

print(f'Total number of rows: {total_row_count}')
print(f'Number of matches: {match_count}')


Total number of rows: 7929
Number of matches: 26964


# Create the XLSX files for each building

In [38]:
import csv
import openpyxl
import os
from collections import defaultdict
from openpyxl.utils import get_column_letter

# Define the file path and project
csv_file = 'Niederscherli_transformed.csv'
project = 'Niederscherli'

def csv_to_xlsx(csv_file, project):
    with open(csv_file, 'r') as csvf:
        reader = csv.reader(csvf)
        headers = next(reader) # get the headers
        data = defaultdict(list)
        for row in reader:
            data[row[headers.index("TIMBER_OS.Gebäudenummer")]].append(row)

    # Make export directory if it doesn't exist
    if not os.path.exists("export"):
        os.makedirs("export")

    for key in data.keys():
        wb = openpyxl.Workbook()
        wb.guess_types = True
        ws = wb.active
        ws.append(headers)  # Write the header in xlsx
        for row in data[key]:  
            ws.append(row)  # Write the remaining rows in xlsx
        # Save the file in the export directory
        filename = f"{project} {key}.xlsx"
        output_file = os.path.join("export", filename)
        wb.save(output_file)
        print(f"Saved file: {filename}")

csv_to_xlsx(csv_file, project)

Saved file: Niederscherli A.xlsx
Saved file: Niederscherli B.xlsx
Saved file: Niederscherli A2.xlsx
Saved file: Niederscherli B2.xlsx
Saved file: Niederscherli A1.xlsx
Saved file: Niederscherli B1.xlsx
Saved file: Niederscherli D.xlsx
Saved file: Niederscherli STA.xlsx
Saved file: Niederscherli C.xlsx
Saved file: Niederscherli .xlsx
Saved file: Niederscherli D1.xlsx
Saved file: Niederscherli D2.xlsx
Saved file: Niederscherli AB.xlsx
Saved file: Niederscherli CD.xlsx
Saved file: Niederscherli BHF.xlsx
Saved file: Niederscherli SPE.xlsx
